In [1]:
#import jax.numpy as np
#from jax import pmap
import numpy as np
from maxnorm.maxnorm_completion import *
from maxnorm.tenalg import *
import sparse
from itertools import product

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

%load_ext autoreload

In [2]:
%autoreload

In [3]:
# create random, low-rank tensor
t = 3
n = 200
r = 4
#delta = 0.1
const = 10
sigma = 0.1
ndata =  const * r * t * n * np.log10(n)
U = [np.random.randn(n, r) for i in range(t)]
print("n data: %.2e" % ndata)
print("n data t/2: %.2e" % int(const * r * n**(t/2) * np.log10(n)))

n data: 5.52e+04
n data t/2: 2.60e+05


In [4]:
max_qnorm_ub(U)

69.35314165950732

In [5]:
observation_mask = sparse.random(tuple([n for i in range(t)]), ndata * n**(-t))
data = observation_mask.copy()

clean_data = kr_get_items(U, data.coords)

#clean_data = np.array(list(map(get_item_coords, [data.coords[:, i] for i in range(data.nnz)])))

# for i in range(data.nnz):
#     pred = get_item(U, data.coords[:, i])
#     data.data[i] = pred
# clean_data = data.data

clean_data_rms = np.sqrt(np.sum(clean_data)**2 / len(clean_data))

data.data = clean_data + np.random.randn(data.nnz) * sigma
print(data.nnz)

55224


In [6]:
print(10* 400**(3/2))
print(10 * r * t * 400)

80000.0
48000


In [7]:
data.coords

array([[  0,   0,   0, ..., 199, 199, 199],
       [  1,   1,   1, ..., 193, 194, 197],
       [ 42,  64, 129, ..., 133,  44, 113]])

In [8]:
clean_data_rmse = np.sqrt(loss(U, data) / data.nnz)
delta = 2 * clean_data_rmse
#np.sqrt(np.sum((data.data - clean_data)**2)) * 2 / np.sqrt(data.nnz) * 1.5

print("rms of data:        %f" % clean_data_rms)
print("delta parameter:    %f" % delta)
print("rmse of U true:     %f" % np.sqrt(loss(U, data) / data.nnz))

/home/kameron/local/anaconda3/lib/python3.7/site-packages/sparse/_coo/numba_extension.py:18: NumbaDeprecationWarning: No direct replacement for 'numba.targets' available. Visit https://gitter.im/numba/numba-dev to request help. Thanks!
  from numba.targets.imputils import impl_ret_borrowed, lower_constant
/home/kameron/local/anaconda3/lib/python3.7/site-packages/sparse/_coo/numba_extension.py:18: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.targets.imputils', please update to use 'numba.core.imputils' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.targets.imputils import impl_ret_borrowed, lower_constant
/home/kameron/local/anaconda3/lib/python3.7/site-packages/sparse/_coo/numba_extension.py:18: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'impl_ret_borrowed' requested from: 'numba.targets.imputils', please u

rms of data:        1.490529
delta parameter:    0.199941
rmse of U true:     0.099970


In [9]:
# from tensorly.contrib.sparse.decomposition import parafac

# mask = data != 0
# core, factors = parafac(data, r, mask=mask, init='random', verbose=True, tol=1e-3, n_iter_max=8)
# scale_mat = np.diag(core.todense()**(1/t))
# U = [factors[i].todense() @ scale_mat for i in range(t)]

In [ ]:
%autoreload
Unew2, cost_arr = tensor_completion_maxnorm(data, r*3, delta * np.sqrt(data.nnz), 
                                            init='svdrand', kappa=100, beta=0.1, tol=1e-4, max_iter=100)

Initial cost: 9089131.382454
Initial qnorm_ub: 1.047040
|| r || = 2.014243, delta = 46.985689
Entering inner loop for factor 0
qnorm_factr = 1.037304
initial objective 9.089131e+06
iteration 30, objective 9.047e+06, relative change 2.477e-04        
maximum iteration 30 reached with relative change in objective function 0.00025
Entering inner loop for factor 1
qnorm_factr = 44.337665
initial objective 9.046947e+06
iteration 30, objective 8.760e+06, relative change 5.198e-04        
maximum iteration 30 reached with relative change in objective function 0.00052
Entering inner loop for factor 2
qnorm_factr = 123.758436
initial objective 8.759672e+06
iteration 30, objective 7.763e+06, relative change 8.955e-04        
maximum iteration 30 reached with relative change in objective function 0.0009

Iteration 0 complete


scaled || r || = 1.876598, delta = 0.199941
Max-qnorm upper bound: 184.383569
Cost function:         7.763e+06


Entering inner loop for factor 0
qnorm_factr = 4.302214
ini

iteration 30, objective 2.603e+06, relative change 1.516e-05        
maximum iteration 30 reached with relative change in objective function 1.5e-05

Iteration 8 complete


scaled || r || = 1.170733, delta = 0.199941
Max-qnorm upper bound: 991.008269
Cost function:         2.603e+06


Entering inner loop for factor 0
qnorm_factr = 9.224434
initial objective 2.603357e+06
iteration 20, objective 2.584e+06, relative change 9.232e-06        
relative change in objective function 9.2e-06 is within tolerance 1e-05 after 20 iterations
Entering inner loop for factor 1
qnorm_factr = 265.235765
initial objective 2.584074e+06
iteration 26, objective 2.565e+06, relative change 9.423e-06        
relative change in objective function 9.4e-06 is within tolerance 1e-05 after 26 iterations
Entering inner loop for factor 2
qnorm_factr = 442.797406
initial objective 2.565411e+06
iteration 30, objective 2.549e+06, relative change 1.404e-05        
maximum iteration 30 reached with relative change in objec

iteration 30, objective 2.375e+06, relative change 9.143e-06        
relative change in objective function 9.1e-06 is within tolerance 1e-05 after 30 iterations
Entering inner loop for factor 2
qnorm_factr = 581.435919
initial objective 2.375229e+06
iteration 23, objective 2.372e+06, relative change 9.480e-06        
relative change in objective function 9.5e-06 is within tolerance 1e-05 after 23 iterations

Iteration 17 complete


scaled || r || = 1.126341, delta = 0.199941
Max-qnorm upper bound: 1695.283370
Cost function:         2.372e+06


Entering inner loop for factor 0
qnorm_factr = 13.362825
initial objective 2.371517e+06
iteration 28, objective 2.368e+06, relative change 9.345e-06        
relative change in objective function 9.3e-06 is within tolerance 1e-05 after 28 iterations
Entering inner loop for factor 1
qnorm_factr = 372.062185
initial objective 2.368034e+06
iteration 29, objective 2.364e+06, relative change 9.017e-06        
relative change in objective function 9e-06

iteration 25, objective 2.304e+06, relative change 9.425e-06        
relative change in objective function 9.4e-06 is within tolerance 1e-05 after 25 iterations
Entering inner loop for factor 1
qnorm_factr = 396.173341
initial objective 2.304268e+06
iteration 24, objective 2.302e+06, relative change 9.601e-06        
relative change in objective function 9.6e-06 is within tolerance 1e-05 after 24 iterations
Entering inner loop for factor 2
qnorm_factr = 680.809248
initial objective 2.302396e+06
iteration 26, objective 2.301e+06, relative change 9.684e-06        
relative change in objective function 9.7e-06 is within tolerance 1e-05 after 26 iterations

Iteration 26 complete


scaled || r || = 1.112315, delta = 0.199941
Max-qnorm upper bound: 2037.402072
Cost function:         2.301e+06


Entering inner loop for factor 0
qnorm_factr = 15.385390
initial objective 2.300642e+06
iteration 24, objective 2.299e+06, relative change 9.647e-06        
relative change in objective function 9.6e-

In [ ]:
#print("loss of U parafac:  %f" % np.sqrt(loss(U,data) / data.nnz))
print("loss of U true:     %f" % np.sqrt(loss(U, data) / data.nnz))
print("loss of U max:      %f" % np.sqrt(loss(Unew2, data) / data.nnz))

In [ ]:
import matplotlib.pyplot as plt

plt.semilogy(cost_arr)
plt.xlabel('iterate')
plt.ylabel('cost')

## Generalization error

In [ ]:
mse_gen2 = kr_dot(Unew2,Unew2) + kr_dot(U,U) - 2 * kr_dot(Unew2, U)
norm_true = kr_dot(U,U)
print("relative RMSE: %1.4e" % np.sqrt(mse_gen2/norm_true))

In [ ]:
fig, axs = plt.subplots(1,t, figsize=(6, 20))
for i in range(t):
    axs[i].imshow(Unew2[i])